In [1]:
%load_ext autoreload
%autoreload 2

from graph import Graph
from nnsight.models import UnifiedTransformer
from transformer_lens import HookedTransformer
from gt_dataset import EAPDataset
from metrics import get_metric
from functools import partial

In [2]:
g = Graph()
g = g.from_json("circuits/greater-than_manual.json")

In [4]:
[g.nodes[f] for f in g.nodes.keys() if "m4." in f]

[Node(m4.0, in_graph: False),
 Node(m4.1, in_graph: False),
 Node(m4.2, in_graph: False),
 Node(m4.3, in_graph: False),
 Node(m4.4, in_graph: False),
 Node(m4.5, in_graph: False),
 Node(m4.6, in_graph: False),
 Node(m4.7, in_graph: False),
 Node(m4.8, in_graph: False),
 Node(m4.9, in_graph: False),
 Node(m4.10, in_graph: False),
 Node(m4.11, in_graph: False),
 Node(m4.12, in_graph: False),
 Node(m4.13, in_graph: False),
 Node(m4.14, in_graph: False),
 Node(m4.15, in_graph: False),
 Node(m4.16, in_graph: False),
 Node(m4.17, in_graph: False),
 Node(m4.18, in_graph: False),
 Node(m4.19, in_graph: False),
 Node(m4.20, in_graph: False),
 Node(m4.21, in_graph: False),
 Node(m4.22, in_graph: False),
 Node(m4.23, in_graph: False),
 Node(m4.24, in_graph: False),
 Node(m4.25, in_graph: False),
 Node(m4.26, in_graph: False),
 Node(m4.27, in_graph: False),
 Node(m4.28, in_graph: False),
 Node(m4.29, in_graph: False),
 Node(m4.30, in_graph: False),
 Node(m4.31, in_graph: False),
 Node(m4.32, in_gr

In [5]:
model = HookedTransformer.from_pretrained("gpt2-small", device="cuda")
model.cfg.use_split_qkv_input = True
model.cfg.use_attn_result = True
model.cfg.use_hook_mlp_in = True

dataset = EAPDataset("greater-than-gpt2.csv")
dataloader = dataset.to_dataloader(16)
metric_fn = get_metric("prob_diff", "greater-than", model.tokenizer, model)

/home/aaron/miniconda3/envs/othello/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded pretrained model gpt2-small into HookedTransformer


In [7]:
from evaluate_graph import evaluate_graph, evaluate_baseline

baseline = evaluate_baseline(model, dataloader, partial(metric_fn, loss=False, mean=False)).mean().item()

results = evaluate_graph(model, g, dataloader, partial(metric_fn, loss=False, mean=False), prune=False).mean().item()

  0%|          | 0/63 [00:00<?, ?it/s]

100%|██████████| 63/63 [00:02<00:00, 29.28it/s]


In [8]:
print(f"Faithfulness: {results / baseline}")

Faithfulness: 0.512998701440054
